
# YOLOv5s ile SKU-110K Eğitimi — Çalıştırılabilir Notebook

Bu notebook SKU-110K veri setini (eg4000/SKU110K_CVPR19) **YOLOv5 (yolov5s)** ile eğitmek için gerekli adımları içerir:
- Gerekli kütüphaneleri yükleme
- Veri setini YOLO formatına dönüştürme (CSV -> `labels/*.txt`)
- `data/sku110k.yaml` oluşturma
- `yolov5s` ile eğitim komutu

**Notlar**
- Bu notebook veri dosyalarını otomatik indirmez (çünkü SKU-110K büyük ve lisans/erişim adımları olabilir). Veri setini yerel olarak `/mnt/data/SKU110K` içine yerleştiriniz veya Roboflow/Kaggle'dan COCO/YOLO formatında indirin.
- Orijin repo (CSV formatında) için referans: `https://github.com/eg4000/SKU110K_CVPR19`. Ultralytics dokümantasyonunda SKU-110K desteği örnekleri vardır.


In [ ]:

# 1) Gerekli kütüphaneleri ve YOLOv5'i kur
# (Çalıştırmadan önce runtime'inizde internet erişimi ve yeterli disk/gpu olduğundan emin olun)

# Clone yolov5 (Ultralytics) - bu repo uzun süredir popülerdir
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

# Alternatif: yeni ultralytics paketini kullanmak isterseniz `pip install ultralytics` seçeneği de var.


In [ ]:

# 2) Veri hazırlık: klasör yapısı beklentisi
# Notebook'un devamında veri setinin şu yapıda olduğunu varsayıyoruz:
# /mnt/data/SKU110K/images/train/*.jpg
# /mnt/data/SKU110K/images/val/*.jpg
# /mnt/data/SKU110K/images/test/*.jpg
# Orijinal eg4000 repo CSV anotasyonlarıysa bu notebook CSV'yi YOLO formatına çeviren bir hücre de içerir.

import os
from pathlib import Path

DATA_ROOT = Path('/mnt/data/SKU110K')  # kullanıcı veri dizinini buraya koymalı
print('DATA_ROOT exists:', DATA_ROOT.exists())


In [ ]:

# 3) (Opsiyonel) CSV -> YOLO (labels .txt) dönüşümü
# Eg4000 repo'da anotasyonlar CSV formatında olur. Bu hücre CSV formatını algılamaya çalışır ve
# YOLOv5 için gerekli her görüntüye karşılık gelen label txt dosyaları oluşturur.
# CSV beklenen sütunlardan herhangi birini kullanacak esnek parser yazıldı.

import csv
import shutil
from PIL import Image

def convert_csv_to_yolo(csv_path, images_dir, labels_dir, classes_map=None):
    """CSV'den YOLO label'larına dönüştürür.
    Beklenen csv satırı: image_filename, x1, y1, x2, y2, [class]
    Koordinatlar piksel cinsindendir.
    classes_map: opsiyonel dict mapping class_name -> class_id
    """
    labels_dir = Path(labels_dir)
    labels_dir.mkdir(parents=True, exist_ok=True)
    images_dir = Path(images_dir)
    # read csv
    with open(csv_path, newline='') as f:
        reader = csv.reader(f)
        header = next(reader)
        # normalize header lower
        header_l = [h.strip().lower() for h in header]
        # try to find column indices
        def find_one(*opts):
            for o in opts:
                if o in header_l:
                    return header_l.index(o)
            return None
        fname_idx = find_one('filename','image','image_name','img')
        x1_idx = find_one('x1','xmin','left')
        y1_idx = find_one('y1','ymin','top')
        x2_idx = find_one('x2','xmax','right')
        y2_idx = find_one('y2','ymax','bottom')
        cls_idx = find_one('class','label','category','category_id','cls')
        if None in [fname_idx,x1_idx,y1_idx,x2_idx,y2_idx]:
            raise ValueError('CSV header colonneleri bulunamadı. Header: {}'.format(header))
        # group boxes by image
        boxes_by_image = {}
        for row in reader:
            fname = row[fname_idx].strip()
            x1 = float(row[x1_idx]); y1 = float(row[y1_idx]); x2 = float(row[x2_idx]); y2 = float(row[y2_idx])
            cls = row[cls_idx].strip() if cls_idx is not None else '0'
            boxes_by_image.setdefault(fname, []).append((x1,y1,x2,y2,cls))
    # convert per image
    for fname, boxes in boxes_by_image.items():
        img_path = images_dir / fname
        if not img_path.exists():
            # try JPEG/JPG alternative or skip
            alt = None
            for ext in ['.jpg','.jpeg','.png','.bmp']:
                if (images_dir / (Path(fname).stem + ext)).exists():
                    alt = images_dir / (Path(fname).stem + ext)
                    break
            if alt:
                img_path = alt
            else:
                # skip missing image
                continue
        w,h = Image.open(img_path).size
        label_fname = labels_dir / (Path(fname).stem + '.txt')
        with open(label_fname, 'w') as lf:
            for x1,y1,x2,y2,cls in boxes:
                # map class
                if classes_map:
                    cls_id = classes_map.get(cls,0)
                else:
                    try:
                        cls_id = int(cls)
                    except:
                        cls_id = 0
                # convert to YOLO format (x_center y_center width height) normalized [0,1]
                xc = ((x1+x2)/2.0)/w
                yc = ((y1+y2)/2.0)/h
                bw = (x2-x1)/w
                bh = (y2-y1)/h
                lf.write(f"{cls_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")
    print('CSV -> YOLO dönüşümü tamamlandı. Label klasörü:', labels_dir)

# Örnek kullanım (CSV'nizi doğru path'e koyun):
# convert_csv_to_yolo('/mnt/data/SKU110K/annotations/annotations_train.csv','/mnt/data/SKU110K/images/train','/mnt/data/SKU110K/labels/train')


In [ ]:

# 4) data YAML dosyası oluşturma (YOLOv5 format)
# `nc` : sınıf sayısı. SKU-110K için araştırmalarda tek sınıf (object) veya çok sınıf çalışmaları var.
# Eğer elinizde class listesi yoksa önce labels klasörünüzdeki maksimum class_id'ye bakın.

from pathlib import Path
import yaml

# örnek paths - kullanıcıya göre özelleştir
data_yaml = {
    'train': '/mnt/data/SKU110K/images/train',
    'val': '/mnt/data/SKU110K/images/val',
    'test': '/mnt/data/SKU110K/images/test',
    'nc': 1,   # eğer çoklu sınıfınız varsa burayı güncelleyin
    'names': ['product']  # eğer çoklu sınıf varsa isimleri buraya koyun
}
Path('data').mkdir(exist_ok=True)
with open('data/sku110k.yaml','w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)
print('data/sku110k.yaml oluşturuldu:
', open('data/sku110k.yaml').read())


In [ ]:

# 5) Eğitim: yolov5s ile temel eğitim komutu
# Aşağıdaki örnekte: img=640, batch=8 (GPU'ya göre arttırın), epochs=50
# Eğer GPU yoksa eğitim çok yavaş olacaktır.

!python train.py --img 640 --batch 8 --epochs 50 --data data/sku110k.yaml --weights yolov5s.pt --name sku110k_yolov5s --cache

# Eğitim çıktılarına `runs/train/sku110k_yolov5s` altında bakabilirsiniz.
